In [ ]:
import time
import sys
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from functools import reduce
from itertools import chain
from scipy.stats import stats
from scipy.stats import rankdata
from scipy.optimize import minimize

from mpl_toolkits.mplot3d import Axes3D

from pyspark.mllib.linalg.distributed import IndexedRowMatrix, IndexedRow
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import create_map, col, to_date, date_format, year, month, dayofmonth, when, lit, lag, array, explode, struct, udf, first
from pyspark.sql.functions import sum as spark_sum, avg as spark_avg, count, stddev as spark_stddev
from pyspark.sql.types import FloatType, StructField, StructType, DateType, IntegerType, ArrayType
from pyspark.sql import SparkSession, Window, DataFrame
from pylab import *
from matplotlib.ticker import LinearLocator, FormatStrFormatter

In [ ]:
pd.set_option('display.max_columns', 10000000)
pd.set_option('display.max_rows', 10000000)
pd.set_option('display.width', 10000000)

------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Defining Spark Session for pseudo-distributed computing:

In [ ]:
spark = SparkSession.builder.appName('Portfolio_Optimization').getOrCreate()
sc = spark.sparkContext
sc

# Loading CSV daily price Funds file.

In [ ]:
portfolio_path_file = 'data.csv'
portfolio_data = spark.read.format("csv").options(header="true").load(portfolio_path_file)

## Change impure schema portfolio input data.
### Defining portfolio dataframe data:

In [ ]:
schema_portfolio = [date_format(
    to_date(col(portfolio_data.columns[0]), 'dd/MM/yyyy'),
    'yyyy-MM-dd').cast('date').alias('operation_date')] + [col(x).cast('float') for x in portfolio_data.columns[1:]]

### Filtering operation dates without nulls:

In [ ]:
portfolio_data_ns = portfolio_data.where(col(portfolio_data.columns[0]).isNotNull())\
                                        .select(schema_portfolio)

portfolio_data_ns.printSchema()

In [ ]:
partition_field_mod1 = ['operation_date']
writing_path_mod1 = '/data/core/fince/data/portfolioOptimization/price_wharehouse_transform/'
print('\nWriting parquets ...\n')
portfolio_data_ns.repartition(1).write.mode('overwrite').parquet(writing_path_mod1, partitionBy=partition_field_mod1)

%time
print('\nSUCCESS \nPARQUET DATA SAVED!')
print('\nNew root path table data:', writing_path_mod1+'operation_date=yyy-MM-dd', '\nparquet chunks portitioned by:', partition_field_mod1)

portfolio_path_parquet = '/data/core/fince/data/portfolioOptimization/price_wharehouse_transform/'
portfolio_df = spark.read.parquet(portfolio_path_parquet)

# Year parameters input array:

In [ ]:
year_param_1, year_param_2 = 2016, 2019
year_array = list(range(year_param_1, year_param_2+1))
print('Year filter array parameters:', year_array)

In [ ]:
portfolio_dates = portfolio_df.select('*',
                                      year("operation_date").alias('year'), 
                                      month("operation_date").alias('month'), 
                                      dayofmonth("operation_date").alias('day'))

-------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Cleaning data, analytic base table structuration.

In [ ]:
def dates_index(dates_list):
    """
    Dates parser function, transform a list of dates in a dictionary
    :param dates_list: list with date values
    :return: parser udf for sequence of dates
    """
    if not isinstance(dates_list, list):
        raise PythagorasUtilsException('Invalid param')

    if len(dates_list) <= 0:
        raise PythagorasUtilsException('Empty param')

    dates_dict = {date: index for index, date in enumerate(dates_list)}
    result = udf(lambda x: dates_dict[x], IntegerType())

    return result

In [ ]:
operation_dates_list = sorted([x.operation_date for x in portfolio_dates.select('operation_date').distinct().collect()])
print("unique dates list:",len(operation_dates_list))

In [ ]:
date_index_udf = dates_index(operation_dates_list)

In [ ]:
debugging_portfolio = portfolio_dates.where(col('year').isin(year_array)).select('*', (date_index_udf(col('operation_date'))).alias('date_id'))
debugging_portfolio.orderBy(col('operation_date')).limit(10).toPandas()

In [ ]:
long_cols = debugging_portfolio.columns[1:-5]
count_by_col = [spark_sum(col(x)).alias(str(x)) for x in long_cols]
aggregate_columns = debugging_portfolio.select(*count_by_col)

In [ ]:
# removing none type data:
null_counts = aggregate_columns.select([count(when(col(c).isNull(), c)).alias(c) for c in aggregate_columns.columns]).collect()[0].asDict()
drop_cols = [k for k, v in null_counts.items() if v > 0]
removed_errors = debugging_portfolio.drop(*drop_cols)

In [ ]:
# removing NaN & fit vectors with no more than 10 NaN's (days):
missing_counter = removed_errors.select([count(when(col(c).isNull(), c)).alias(c) for c in removed_errors.columns]).collect()[0].asDict()
drop_rude_missing = [k for k, v in missing_counter.items() if v > 10]
remove_rude_missing = removed_errors.drop(*drop_rude_missing)

In [ ]:
numerical_fields = remove_rude_missing.agg(*(spark_avg(c).alias(c) for c in remove_rude_missing.columns if c not in ['operation_date']))
purifying_portfolio = remove_rude_missing.na.fill(numerical_fields.first().asDict())

In [ ]:
w = Window.orderBy("operation_date")
yield_cols = purifying_portfolio.columns[:-5]
yield_portfolio = (reduce(lambda r_df, col_name: r_df.withColumn(col_name, (lag(r_df[col_name]).over(w) / r_df[col_name])-1), yield_cols, purifying_portfolio))\
                                                     .where(col(yield_cols[0]).isNotNull())

# Writing Portfolio's Yield dataframe.

In [ ]:
partition_field_mod2 = ['operation_date']
writing_path_mod2 = '/data/core/fince/data/portfolioOptimization/portfolio_yield_window/'

print('\nWriting parquets ...')
yield_portfolio.repartition(5).write.mode('overwrite').parquet(writing_path_mod2, partitionBy=partition_field_mod2)

%time
print('\nSUCCESS \nPARQUET DATA SAVED!')
print('\nNew root path tabla data:', writing_path_mod2 + 'operation_date=yyy-MM-dd', '\nparquet chunks portitioned by:', partition_field_mod2)

# Reading persisted Portfolio Yields dataframe:

In [ ]:
portfolio_yield_window_path = '/data/core/fince/data/portfolioOptimization/portfolio_yield_window/'
portfolio_yield_df = spark.read.parquet(portfolio_yield_window_path)

In [ ]:
portfolio_yield_df.limit(10).toPandas()

In [ ]:
dataframes = [portfolio_yield_df.select(lit(fund).alias('fund_name'), col(fund).alias('fund_yield')) for fund in portfolio_yield_df.columns[:-1]]

In [ ]:
def unionAll_df(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [ ]:
portfolio_yield_T = unionAll_df(*dataframes).cache()

# Writing Portfolio's Yield Transpose dataframe.

In [ ]:
writing_path_mod3 = '/data/core/fince/data/portfolioOptimization/portfolio_yield_transpose/'

print('\nWriting parquets ...')
portfolio_yield_T.repartition(1).write.mode('overwrite').parquet(writing_path_mod3)

%time
print('\nSUCCESS \nPARQUET DATA SAVED!')
print('\nNew root path tabla data:', writing_path_mod3)

# Reading persisted Portfolio Yields Transpose.

In [ ]:
portfolio_yield_T_path = '/data/core/fince/data/portfolioOptimization/portfolio_yield_transpose/'
portfolio_yield_T_df = spark.read.parquet(portfolio_yield_T_path)

In [ ]:
portfolio_yield_T_df.show()

In [ ]:
sharpe_ratio_df = portfolio_yield_T_df.groupBy("fund_name")\
                                      .agg(spark_avg('fund_yield'), spark_stddev('fund_yield'))\
                                      .select("*", (col("avg(fund_yield)") / col("stddev_samp(fund_yield)")).alias("sharpe_ratio"))\
                                      .orderBy(col("sharpe_ratio").desc())\
                                      .drop("avg(fund_yield)", "stddev_samp(fund_yield)")

# Sharpe ratio:

In [ ]:
sharpe_ratio_df.show(100)

In [ ]:
field_array = portfolio_yield_df.columns[:-1]
monthly_return = np.array(portfolio_yield_df.select(*field_array).collect())
print('test with', len(field_array), 'funds')

In [ ]:
print('monthly_return matrix:\n', monthly_return)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Single Value Decomposition analysis.

In [ ]:
monthly_return_rdd = sc.parallelize(monthly_return.tolist()).zipWithIndex()

# Obtaining model parameters:
n = monthly_return_rdd.count()
p = len(monthly_return_rdd.take(1)[0][0])

In [ ]:
udf_dense_vector = udf(lambda x: Vectors.dense(x), VectorUDT())

In [ ]:
monthly_return_df = spark.createDataFrame(monthly_return_rdd).toDF('features', 'id')
monthly_return_df.select('features').collect()
#monthly_return_df.show()

In [ ]:
monthly_return_df = monthly_return_df.withColumn("features", udf_dense_vector("features"))
monthly_return_df.select('features').collect()#.show()

In [ ]:
stdScaler = StandardScaler(withMean=True, withStd=True, inputCol="features", outputCol="scaled_features")
model = stdScaler.fit(monthly_return_df)

In [ ]:
monthly_return_std_df = model.transform(monthly_return_df).drop("features").withColumnRenamed("scaled_features","features")
monthly_return_std_df.select('features').collect()#.show()

In [ ]:
# Index Row Matrix
monthly_return_irm = IndexedRowMatrix(monthly_return_std_df.rdd.map(lambda x: IndexedRow(x[0], x[1].tolist())))

In [ ]:
SVD = monthly_return_irm.computeSVD(p, True)
U = SVD.U
S = SVD.s.toArray()

In [ ]:
print(S)

In [ ]:
eigen_vals = S**2/(n-1)
eigvals = np.flipud(np.sort(eigen_vals))
cumsum = eigvals.cumsum()
total_variance_explained = cumsum/eigvals.sum()

In [ ]:
print('\nvector of eigenvalues:\n', eigen_vals)
print('\nvector of eigvals:\n', eigvals)
print('\nvector of cumsum:\n', cumsum)
print('\nvector of total_variance_explained:\n', total_variance_explained)

In [ ]:
K = np.argmax(total_variance_explained>0.95)+1
V = SVD.V
U = U.rows.map(lambda x: (x.index, x.vector[0:K]*S[0:K]))

In [ ]:
V

In [ ]:
princ_comps = np.array(list(map(lambda x:x[1], sorted(U.collect(), key = lambda x:x[0]))))
print(princ_comps)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.scatter(princ_comps, princ_comps)

In [ ]:
#setosa = princ_comps[iris_target==0]
#versicolor = princ_comps[iris_target==1]
#verginica = princ_comps[iris_target==2]
#plt.scatter(princ_comps, princ_comps)
#plt.scatter(princ_comps, princ_comps, c="b",label="hola")
#plt.scatter(versicolor[:,0], versicolor[:,1], c="g",label="versicolor")
#plt.scatter(verginica[:,0], verginica[:,1], c="r",label="verginica")

-------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
size = monthly_return.shape[0]
N = monthly_return.shape[1]
T1 = 11
start_month = T1 + 1
T2 = size - start_month
end_month = size
covmatr = np.zeros((N, N))
w_RP = np.zeros((T2, N))

In [ ]:
ret = monthly_return.T
w_EW = np.zeros((T2, N))
onen = np.full((1, N), 1/N)
r_ew  = np .zeros((T2, N))
r_rp = np.zeros((T2, 1))
retEW = np.zeros((T2, 1))
retRP = np.zeros((T2, 1))

In [ ]:
print('Generating optimization parameters...\n')
for y in range(start_month, end_month):
    w_EW[:] = onen
    r_ew[y - start_month] = np.dot(monthly_return[y,:] , 1/N)
    retEW[y - start_month] = sum(r_ew[y-start_month])
%time
print('\nDONE!')

In [ ]:
print('Generating Marginal Risk Contribution variables...\n')
for w in range(start_month, end_month):
    covmatr = np.cov(ret[:,w-T1:w])    
%time
print('\nDONE!')
print('\nvariance & covariance matrix:')
print(covmatr)

# Generator function for Risk Contribution variables
- mrc aka: marginal risk contribution
- rc aka: risk contribution

In [ ]:
def RC(weight, covmatr):
    weight = np.array(weight)
    variance = weight.T @ covmatr @ weight
    sigma = variance ** .5
    mrc = 1/sigma * (covmatr @ weight)
    rc = weight * mrc
    rc = rc/rc.sum()
    return rc

# Generator function for RiskParity objective variables

In [ ]:
def RiskParity_objective(x):
    variance = x.T @ covmatr @ x
    sigma = variance ** .5
    mrc = 1/sigma * (covmatr @ x)
    rc = x * mrc
    a = np.reshape(rc, (len(rc),1))
    risk_diffs = a - a.T
    sum_risk_diffs_squared = np.sum(np.square(np.ravel(risk_diffs)))
    return sum_risk_diffs_squared

In [ ]:
def weight_sum_constraint(x):
    return np.sum(x) - 1.0
        
def weight_longonly(x):
    return x

# Function object for instance on Minimization scipy function

In [ ]:
def RiskParity(covmatr):
    x0 = np.repeat(1/covmatr.shape[1], covmatr.shape[1])
    constraints = ({'type': 'eq', 'fun': weight_sum_constraint},
                   {'type': 'ineq', 'fun' : weight_longonly})
    options = {'ftol' : 1e-20, 'maxiter': 999}
    result = minimize(fun = RiskParity_objective,
                      x0 = x0,
                      constraints = constraints,
                      options = options)
    return result.x

In [ ]:
print('Generating optimized return matrices...')
for w in range(start_month, end_month):
    w_RP[w - start_month] = RiskParity(covmatr)
    r_rp[w - start_month] = np.dot(monthly_return[w,:], w_RP[w - start_month,:])
    retRP[w - start_month] = sum(r_rp[w - start_month])
%time
print('\nDONE!')
print('\nw_RP matrix:')
print(w_RP)
print('\nretRP matrix:')
print(retRP)

# El eje x será el mes, el eje y será el activo (fondo), y z será el peso activo del portafolio.

In [ ]:
mx = np.amax(w_RP)
mn = np.amin(w_RP)

fig = plt.figure()
ax = fig.gca(projection = '3d')

X = np.arange (0, T2, 1)
Y = np.arange( 0, N, 1)
X, Y = np.meshgrid(X, Y)
Z = np.transpose(w_RP)

surf = ax.plot_surface(X, Y, Z, cmap = cm.Reds_r, linewidth = 0)

ax.set_zlim(mn-.02, mx+.05)
plt.show()